# Homework 4: Multiple Linear Regression
***

**Name**: 

***

This assignment is due on Gradescope by **Friday February 21st at 5:00PM**. If you submit the assignment  by this deadline, you will receive 2 bonus points. If you need a little extra time, you may submit your work by **Monday February 24th at 5:00PM**. Your solutions to theoretical questions should be done in Markdown directly below the associated question.  Your solutions to computational questions should include any specified R code and results as well as written commentary on your conclusions.  Remember that you are encouraged to discuss the problems with your classmates, but **you must write all code and solutions on your own**.

**NOTES**: 

- There are 2 total questions on this assignment. 
- If you're not familiar with typesetting math directly into Markdown then by all means, do your work on paper first and then typeset it later.  Remember that there is a [reference guide](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) linked here. **All** of your written commentary, justifications and mathematical work should be in Markdown.
- Because you can technically evaluate notebook cells in a non-linear order, it's a good idea to do Kernel $\rightarrow$ Restart & Run All as a check before submitting your solutions.  That way if we need to run your code you will know that it will work as expected. 
- It is **bad form** to make your reader interpret numerical output from your code.  If a question asks you to compute some value from the data you should show your code output **AND** write a summary of the results in Markdown directly below your code. 
- This probably goes without saying, but... For any question that asks you to calculate something, you **must show all work and justify your answers to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit. 



---

### Problem 1 <span style="color: #CFB87C;">(STAT 5010 Students Only)</span> (50 Points) Trace of the hat matrix

Define the trace of a square matrix 

$A = \left(\begin{matrix}
a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{n,1} & a_{n,2} & \cdots & a_{n,n} 
\end{matrix} \right)$

to be $tr(A) = \sum^n_{i=1}a_{i,i}$, i.e., the sum of the diagonal elements of $A$.

**PART A:** Let $B$ be a $m \times n$ matrix and $C$ $n\times m$ matrix. Prove that $tr(BC) = tr(CB).$

**Answer**

We can achieve this by first looking at individual diagonal elements from the same level in the two matrx multiplications. Let $D = BC$ and $E = CB$. Then by definition

$$d_{i, i} = \sum_{j=1}^nb_{i, j}c_{j, i} \text{ for $i \in {1, 2, \cdots, m}$}$$
$$e_{i, i} = \sum_{j=1}^mc_{i, j}b_{j, i} \text{ for $i \in {1, 2, \cdots, n}$}$$

By definition $tr(A) = \sum^n_{i=1}a_{i,i}$, so

$$tr(D) = \sum_{i=1}^md_{i, i} = \sum_{i=1}^m\sum_{j=1}^nb_{i, j}c_{j, i}$$
$$tr(E) = \sum_{i=1}^ne_{i, i} = \sum_{i=1}^n\sum_{j=1}^mc_{i, j}b_{j, i}$$

Swap $j$ and $i$ for $tr(E)$ we have

$$tr(E) = \sum_{j=1}^ne_{j, j} = \sum_{j=1}^n\sum_{i=1}^mc_{j, i}b_{i, j} = \sum_{i=1}^m\sum_{j=1}^nb_{i, j}c_{j, i} = tr(D)$$

**PART B:** Let $H$ be the hat matrix as defined in class. Prove that $tr(H) = p+1$, where $p$ is the number of parameters in the regression.

**Answer**

$$\begin{align*}tr(H) &= tr(X(X^TX)^{-1}X^T) \\
                        &= tr(X(X^TX)^{-1}X^T) \\
                        &= tr((X(X^TX)^{-1})X^T)) \\
                        &= tr(X^T(X(X^TX)^{-1}))) \\
                        &= tr((X^TX)(X^TX)^{-1})) \\
                        &= tr((X^T_{(p+1) \times n}X_{n \times (p+1)})(X^T_{(p+1) \times n}X_{n \times (p+1)})^{-1}) \text{ because we have p features plus 1 bias term}\\
                        &= tr((X^TX)_{(p+1) \times (p+1)}(X^TX)^{-1}_{(p+1) \times (p+1)})\\
                        &= tr(I_{(p+1) \times (p+1)})\\
                        &= p+1\\
\end{align*}$$

---

### Problem 2 - Multiple Linear Regression and Model Selection (50 points)

We will further examine the `Fish.csv` dataset in this problem. 

"This dataset is a record of 7 common different fish species in fish market sales. With this dataset, a predictive model can be performed using machine friendly data and estimate the weight of fish can be predicted."

**Response**:
- Weight (in grams)

**Features**:
- Length1 (vertical length in cm)
- Length2 (diagonal length in cm)
- Length3 (cross length in cm)
- Height (in cm)
- Width (diagonal width in cm)

The species name of the fish is also given. 

**Part A**: Read the data from the csv.  As you are reading in `Fish.csv`, drop the species column as it is non-numerical.

Also, make sure to re-order the columns so that the response variable is the last column.

In [6]:
# Read the CSV file
fish <- read.csv("Fish.csv")

# Drop the "species" column
library(dplyr)
fish <- fish %>% select(-Species)

# View the resulting data
head(fish)

,Weight,Length1,Length2,Length3,Height,Width
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,242,23.2,25.4,30.0,11.5200,4.0200
2,290,24.0,26.3,31.2,12.4800,4.3056
3,340,23.9,26.5,31.1,12.3778,4.6961
4,363,26.3,29.0,33.5,12.7300,4.4555
5,430,26.5,29.0,34.0,12.4440,5.1340
6,450,26.8,29.7,34.7,13.6024,4.9274


**Part B:** Fit a multiple linear regression model to the data.

- print the regression coeficients to the screen.

- Use a Markdown cell to specify the MLR model in the form: $ \hat{y} = \beta_0+\beta_1 X_1 + \beta_2 X_2 + ... + \beta_p X_p $

In [15]:
model <- lm(Weight ~ Length1 + Length2 + Length3 + Height + Width, data = fish)

# View the model summary to get details about the regression coefficients, R-squared, etc.
summary(model)


Call:
lm(formula = Weight ~ Length1 + Length2 + Length3 + Height + 
    Width, data = fish)

Residuals:
    Min      1Q  Median      3Q     Max 
-243.69  -65.10  -25.52   57.98  447.25 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -499.587     29.572 -16.894  < 2e-16 ***
Length1       62.355     40.209   1.551  0.12302    
Length2       -6.527     41.759  -0.156  0.87601    
Length3      -29.026     17.353  -1.673  0.09643 .  
Height        28.297      8.729   3.242  0.00146 ** 
Width         22.473     20.372   1.103  0.27169    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 123.2 on 153 degrees of freedom
Multiple R-squared:  0.8853,	Adjusted R-squared:  0.8815 
F-statistic: 236.2 on 5 and 153 DF,  p-value: < 2.2e-16


**Answer** 

The MLR model is: $\hat{Weight} = -499.587+62.355 Length1 -6.527 Length2 + -29.026 Length3 + 28.297 Height + 22.473 Width$

**Part C**: Perform the appropriate statistical hypothesis test at the $\alpha = 0.01$ significance level to determine if _at least one_ of the features is related to the the response $y$.  

**Answer**

The p-value for Height is 0.00146 < 0.01 and passed the hypothesis test.

**Part D**: Write a function `forward_select(df, resp_str, maxk)` that takes in the DataFrame, the name of the column corresponding to the response, and the maximum number of desired features, and returns a list of feature names corresponding to the `maxk` most important features via forward selection.  At each stage in forward selection you should add the feature whose inclusion in the model would result in the lowest sum of squared errors $(SSE)$. Use your function to determine the best $k=3$ features to include in the model. Clearly indicate which feature was added in each stage. 

**Note**: The point of this exercise is to see if you can implement **foward_select** yourself.  You may not call any R method that explicitly performs forward selection.

In [66]:
forward_select <- function(df, resp_str, maxk) {

    res_str <- character(0)

    for (i in 1:maxk){
        if (length(res_str) == 0) {
            model <- lm(Weight ~ 1, data = df)
        } else {
            formula <- as.formula(paste("Weight ~", paste(res_str, collapse = " + ")))
            model <- lm(formula, data = df)
        }
        residuals <- model$residuals
        SSE <- sum(residuals^2)

        best_SSE_improvement <- 0
        best_col <- ""

        for (colname in resp_str) {
            cols <- c(res_str, colname)
            formula <- as.formula(paste("Weight ~", paste(cols, collapse = " + ")))
            model <- lm(formula, data = df)
            residuals_curr <- model$residuals
            SSE_curr <- sum(residuals_curr^2)
            if (SSE - SSE_curr > best_SSE_improvement) {
                best_SSE_improvement <- SSE - SSE_curr
                best_col <- colname
            }
        }
        res_str <- c(res_str, best_col)
    }

    return(res_str)
}

**Part E**: Write down the reduced multiple linear regression model, including estimated parameters, obtained by your forward selection process. 

In [67]:
# Get the column names of the dataframe and store them in a vector
col_vector <- colnames(fish)

# Remove 'Weight' from the vector (without affecting the dataframe)
resp_str <- col_vector[col_vector != "Weight"]

maxk <- 3
selected_colnames <- forward_select(fish, resp_str, maxk)
formula <- as.formula(paste("Weight ~", paste(selected_colnames, collapse = " + ")))
model <- lm(formula, data = fish)

summary(model)


Call:
lm(formula = formula, data = fish)

Residuals:
    Min      1Q  Median      3Q     Max 
-246.79  -77.57  -33.26   82.47  453.51 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -521.000     29.331 -17.763  < 2e-16 ***
Length3       19.445      1.812  10.728  < 2e-16 ***
Width         62.833     14.560   4.315 2.83e-05 ***
Height         3.853      3.849   1.001    0.318    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 126.3 on 155 degrees of freedom
Multiple R-squared:  0.8778,	Adjusted R-squared:  0.8755 
F-statistic: 371.3 on 3 and 155 DF,  p-value: < 2.2e-16


**Answer** 

The MLR model is: $\hat{Weight} = -521+19.445 Length3 + 62.833 Width + 3.853 Height$